# Der Bibelcode

### Einführung


1997 hat ein Buch des Journalisten Michael Drosnin grosse Aufmerksamkeit erregt. Das Buch mit dem Namen *Bible code* postuliert, dass in der Bibel (genauer gesagt in der Tora) versteckte Botschaften codiert sind. Diese Botschaften sind durch equidistante Folgen von Zeichen codiert, wie im folgenden Bild gezeigt ist (Quelle: Wikipedia)
<figure style="align:center">
    <img src="images/bible-code.png" width=200px/>
</figure>

In dieser Fallstudie werden wir überprüfen, ob wir auch in anderen Werken solche Botschaften finden. Wir nutzen dazu das Buch Krieg und Frieden von Leo Tolstoy, welches frei auf [Projekt Gutenberg](https://www.gutenberg.org) verfügbar ist. 

### Aufgabe

Unser Ziel ist es in einem Text alle als *Bible code* versteckten Wörter zu finden welche in einem Wörterbuch vorkommen. Dabei ist der Text in einer einfachen `.txt`-Datei gegeben und das Wörterbuch als eine `.json`-Datei.

### Problem Analyse und Zerlegung

Wir wollen als erstes das ganze Problem in Teilprobleme zerlegen. Dabei sollen die Teilprobleme einfach zu implementieren sein und sich auch gut zur gesamten Lösung zusammensetzten lassen.

Überlegen Sie sich Teilprobleme und ergänzen Sie die Liste damit. Überlegen Sie sich auch, welche Teilprobleme Sie einfach als Funktion implementieren können:

- Einlesen des Textes aus einer .txt-Datei.
- Einlesen eines Wörterbuches aus einer .json-Datei.
- ...
- *Entfernen von Sondernzeichen aus einem Text (und umwandeln in Kleinbuchstaben).*
- *Extrahieren eines Wortes aus einem Text an einer Stelle mit gegebener Distanz zwischen den Buchstaben.*
- *Stellen finden wo der Anfangsbuchstabe des Wortes vorkommt.*
- *Testen von allen Worten im Dictionary.*
- *Speichern einer Stelle wo ein Wort gefunden wurde.*
- *Ausgabe aller Stellen wo ein Wort gefunden wurde.*
- *Ausgabe wie oft ein Wort gefunden wurde.*

### Vorbereiten der Daten

Als Erstes wollen wir die Vorverarbeitung der Daten erstellen. Wir wollen unsere Chancen auf einen Treffer erhöhen indem wir von einem Text möglichst nur die Buchstaben behalten. Dazu schreiben wir uns eine erste Hilfsfunktion. Diese soll: 
- auf Basis eines übergebenen Textes einen neuen Text erstellen.
- möglichst nur Buchstaben behalten.
- die Buchstaben in Kleinbuchstaben umwandeln.

In [1]:
#lower_case_letters = [chr(ord('a')+x) for x in range(0,26)]
#lower_case_letters = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
#c.isalpha()

def clean_text(text): # clean_text as new name? (it is anyway not an appropriate name)
    s = "" 
    for c in text:
        # lower_case = c.lower()
        # if c.lower() in lower_case_letters:
        # if c.isalpha():
        if not c in ['.', ',', '?', '!', ' ']: # ,'\n', '\r', '\t', '-', ':', ';', '*',
            s += c.lower()
    return s

Da wir all unseren Code den wir schreiben immer auch testen sollen, rufen wir die Funktion mit einem einfachen Text auf. Wir schauen dann ob die Funktion das tut was wir erwarten.

In [2]:
clean_text("Eine Frage? ein Ausrufezeichen! und ein Punkt.")

'einefrageeinausrufezeichenundeinpunkt'

### Text laden
Als nächstes wollen wir das laden des Textes aus einem File implementieren. Auch dazu schreiben wir uns eine Funktion welche den Inhalt des Files `war-and-peace.txt` laden soll. Der Text soll dann gleich noch von Sonderzeichen befreit werden.

In [3]:
def load_preprocessed_text():
    f = open("war-and-peace.txt", 'r')
    text = f.read()
    f.close()
    return clean_text(text)

Test wir doch auch diese Funktion gleich indem wir die ersten 100 Zeichen ausgeben.

In [4]:
load_preprocessed_text()[0:100] # Wir geben nur die ersten 100 Zeichen aus. 

'ï»¿theprojectgutenbergebookofwarandpeacebyleotolstoy\n\nthisebookisfortheuseofanyoneanywhereatnocostan'

### Äquidistante Buchstaben
Ein weiteres Teilproblem ist, dass wir an einer Stelle eines Textes ein Wort extrahieren können mit einer gegebener Distanz der Buchstaben. Dafür schreiben wir uns eine Funktion, welche:
- ab einer gegebenen Position `start_position` Buchstaben aus dem Text `text` extrahiert.
- wenn möglich `number_of_letters` Buchstaben mit Distanz `distance` extrahiert.
- die extrahierten Buchstaben als neues Wort zurückgibt.

In [5]:
def equidistant_letter_seq(text, start_position, number_of_letters, distance):
    s = "" 
    end_position = start_position+number_of_letters*distance
    end_position = min(end_position,len(text))
    for i in range(start_position,end_position,distance):
        s += text[i]
    return s

    #i = 0
    #while start_position + i * distance < len(text) and i < number_of_letters:
    #    s += text[start_position + i * distance]
    #    i += 1
    #return s

Testen! Ja, auch diese Funktion testen wir.

In [6]:
equidistant_letter_seq("Der Mond ist aufgegangen", 2, 5, 3)

'ro tu'

### Mögliche Startpositionen

Nun können wir durch einen Text gehen, und schauen wo der Anfangsbuchstaben eines zu suchenden Wortes vorkommt. So wissen wir wo im Text wir überhaupt suchen müssen.

In [7]:
def locate_char_in_text(text, character):
    possible_start_locations = []
    for i in range(0,len(text)):
        if text[i] == character:
            possible_start_locations.append(i)
    return possible_start_locations

Auch diese Funktion können wir ganz einfach kurz testen.

In [8]:
locate_char_in_text("Abba bringt brandneue Tonbändchen auf den Markt.", 'b')

[1, 2, 5, 12, 25]

### Ist das Wort gefunden?

Als nächstes wollen wir in einem Text nachschauen ob an einer bestimmten Position mit einer bestimmten Distanz zwischen den Buchstaben ein Wort vorkommt.

In [9]:
def word_is_found(text, word_to_search, start_pos, distance):
    word_length = len(word_to_search)
    word_in_text = equidistant_letter_seq(text, start_pos, word_length, distance)
    return word_in_text == word_to_search

Genau - wir wollen die Funktion testen.

In [10]:
word_is_found("foo p y t h o n bar", "python", 4, 2)

True

### Der Fundort
Wenn wir ein Wort gefunden haben, dann wollen wir uns dafür ein paar Dinge merken. Dafür verwenden wir eine ganz simple Klasse. Die Klasse soll das gefundene Wort, die Anfangsposition sowie die Distanz zwischen den Buchstaben speichern. Zudem soll die Klasse sich noch einfach über eine `print` Methode auf die Konsole ausgeben lassen.

In [11]:
class WordOccurrence:
    def __init__(self, word, pos, distance):
        self.word = word
        self.pos = pos
        self.distance = distance
        
    def print(self):
        print("found word \"{}\" at position {} with distance {}".format(self.word, self.pos, self.distance))

Testen? - Na gut ....

In [12]:
match = WordOccurrence("love",7,42)
match.print()

found word "love" at position 7 with distance 42


### Suchen
Jetzt können wir viele unserer Teile zusammenfügen und in einem Text alle vorkommen eines Wortes mit einer bestimmten Distanz zwischen den Buchstaben suchen.

In [13]:
def find_word_with_distance(text, word_to_search, distance):
    occurences = []
    possible_starting_locations = locate_char_in_text(text, word_to_search[0])
    for start_position in possible_starting_locations:
        if word_is_found(text, word_to_search, start_position, distance):
            occurences.append(WordOccurrence(word_to_search, start_position, distance))
    return occurences

Ok - sag nichts! Ich teste ja schon...

In [14]:
matches = find_word_with_distance("ein Text mit kleinen Geheimnissen", "emknG", 4)
matches[0].print()

found word "emknG" at position 5 with distance 4


### Ausgabe

Wir wollen noch die Ausgabe von einer Liste von gefundenen Wörtern vereinfachen. Dazu schreiben wir, genau, auch wider eine Funktion.

In [15]:
def print_occurences(found_occurences):
    print("{} occurences found".format(len(found_occurences)))
    for word_info in found_occurences:
        word_info.print()


Testen obligatorisch!

In [16]:
print_occurences(matches)

1 occurences found
found word "emknG" at position 5 with distance 4


### Die erste Anwendung
Endlich können wir nun auch die Anwendung schreiben und nachschauen ob `corona` im text vorkommt.

In [17]:
text = load_preprocessed_text()
word_to_find = "corona"
distance = 2
found_words = find_word_with_distance(text, word_to_find, distance)
print_occurences(found_words)


0 occurences found


Wir können natürlich nicht nur nach Distanz 2, sondern nach beliebigen Distanzen suchen.

In [18]:
text = load_preprocessed_text()
word_to_find = "corona"
#word_to_find = "vetter"
#word_to_find = "luethi"
#word_to_find = "morel"
for dist in range(2,30):
    found_words = find_word_with_distance(text, word_to_find, dist)
    print_occurences(found_words)


0 occurences found
0 occurences found
0 occurences found
0 occurences found
0 occurences found
0 occurences found
0 occurences found
0 occurences found
0 occurences found
0 occurences found
0 occurences found
0 occurences found
0 occurences found
0 occurences found
0 occurences found
0 occurences found
0 occurences found
1 occurences found
found word "corona" at position 2493083 with distance 19
0 occurences found
0 occurences found
1 occurences found
found word "corona" at position 1207766 with distance 22
0 occurences found
0 occurences found
0 occurences found
0 occurences found
0 occurences found
0 occurences found
0 occurences found


## Die letzten Schritte

Zuletzt interessiert es uns noch, was es denn überhaupt alles für Wörter gibt die als solche Equidistant Letter Sequences vorkommen. Dazu müssen wir ein Dictionary laden können. 

In [19]:
def load_dictionary():
    import json
    f = open("dictionary.json", "r")
    dictionary = json.load(f)
    f.close()
    return dictionary

Wir schreiben uns nun noch eine etwas andere Suchfunktion um diese Aufgabe effizienter zu lösen.

In [20]:
def find_all_words(text, dictionary, word_length, distance):
    occurrences = []
    for start_position in range(0,len(text)):
        word = equidistant_letter_seq(text, start_position, word_length, distance)
        if len(word) == word_length and word in dictionary:
            occurrences.append(WordOccurrence(word, start_position, distance))
    return occurrences

Und die Finale Anwendung die uns alle gefundenen Wörter welche in einem Dictionary stehen rausschreibt.

In [21]:
text = load_preprocessed_text()
dictionary = load_dictionary()
for l in range(7, 12):
    for d in range(3, 10):
        print("checking words of length {} with distance {} ...".format(l,d))
        found_words = find_all_words(text, dictionary, l, d)
        print_occurences(found_words)


checking words of length 7 with distance 3 ...
47 occurences found
found word "altaian" at position 100008 with distance 3
found word "trenail" at position 195355 with distance 3
found word "heronry" at position 208074 with distance 3
found word "operant" at position 246708 with distance 3
found word "amenage" at position 364278 with distance 3
found word "sunniah" at position 398706 with distance 3
found word "shatter" at position 399362 with distance 3
found word "nowhere" at position 406180 with distance 3
found word "rosette" at position 406449 with distance 3
found word "lernean" at position 455934 with distance 3
found word "seedman" at position 476492 with distance 3
found word "roedeer" at position 562206 with distance 3
found word "sanicle" at position 607244 with distance 3
found word "spitted" at position 694525 with distance 3
found word "reorder" at position 788536 with distance 3
found word "fluegel" at position 830085 with distance 3
found word "sistine" at position 9277

67 occurences found
found word "shittle" at position 25381 with distance 6
found word "puttier" at position 62020 with distance 6
found word "thistle" at position 84105 with distance 6
found word "deflate" at position 112170 with distance 6
found word "dottard" at position 128236 with distance 6
found word "terreen" at position 178021 with distance 6
found word "bounder" at position 440023 with distance 6
found word "triolet" at position 485882 with distance 6
found word "ichnite" at position 490143 with distance 6
found word "witness" at position 508107 with distance 6
found word "ternion" at position 532744 with distance 6
found word "unclose" at position 582270 with distance 6
found word "trehala" at position 585629 with distance 6
found word "sarcous" at position 638686 with distance 6
found word "dissect" at position 769915 with distance 6
found word "niggler" at position 772675 with distance 6
found word "heather" at position 789757 with distance 6
found word "entheat" at positio

58 occurences found
found word "readmit" at position 40641 with distance 8
found word "shearer" at position 72300 with distance 8
found word "trendle" at position 270096 with distance 8
found word "attempt" at position 300711 with distance 8
found word "reenact" at position 337484 with distance 8
found word "whereso" at position 360450 with distance 8
found word "lactate" at position 367848 with distance 8
found word "stalder" at position 378843 with distance 8
found word "seriate" at position 402163 with distance 8
found word "melting" at position 438434 with distance 8
found word "chogset" at position 448835 with distance 8
found word "etesian" at position 459073 with distance 8
found word "savanna" at position 507835 with distance 8
found word "iterate" at position 515779 with distance 8
found word "resinic" at position 650279 with distance 8
found word "idioted" at position 692385 with distance 8
found word "dissite" at position 705034 with distance 8
found word "sharded" at positi

3 occurences found
found word "terzetto" at position 1651408 with distance 6
found word "shekinah" at position 1974959 with distance 6
found word "traiteur" at position 2599228 with distance 6
checking words of length 8 with distance 7 ...
0 occurences found
checking words of length 8 with distance 8 ...
4 occurences found
found word "mootable" at position 197271 with distance 8
found word "geminate" at position 204999 with distance 8
found word "smelting" at position 438426 with distance 8
found word "aposteme" at position 1782739 with distance 8
checking words of length 8 with distance 9 ...
4 occurences found
found word "vegetate" at position 581087 with distance 9
found word "utterest" at position 1073577 with distance 9
found word "eversion" at position 1346131 with distance 9
found word "eerisome" at position 1564702 with distance 9
checking words of length 9 with distance 3 ...
0 occurences found
checking words of length 9 with distance 4 ...
1 occurences found
found word "edito